In [1]:
import ee
import geemap
import math

ModuleNotFoundError: No module named 'StringIO'

In [6]:
!pip install geemap --user --no-warn-script-location

In [12]:
!earthengine authenticate

Fetching credentials using gcloud
gcloud failed. Please check for any errors above and install gcloud if needed.

'gcloud' is not recognized as an internal or external command,


operable program or batch file.


In [11]:
pip install gcloud


  Created wheel for gcloud: filename=gcloud-0.18.3-py3-none-any.whl size=602937 sha256=dc2774b894d4f656173a3c5b72189098318f987c5752f43503c0044b497579a3
  Stored in directory: c:\users\crort\appdata\local\pip\cache\wheels\86\05\94\4ad6324debd4e3edba7fdf252f1ff2d2aba09673a794e67a27
Successfully built gcloud


In [8]:
#Initialize Google Earth Engine
#ee.Authenticate() just needed the 1st time
ee.Initialize()

EEException: Please authorize access to your Earth Engine account by running

earthengine authenticate

in your command line, and then retry.

In [3]:
#Data loads

#loads feature collection data from Google Earth Engine - We can also upload other feature collections
counties = ee.FeatureCollection("TIGER/2018/Counties")

#filter LA County
la_county = counties.filter(ee.Filter.eq('NAME', 'Los Angeles'))

In [4]:
#loads cities data - Not in current use

#load California Cities Shapefile
cities = ee.FeatureCollection("projects/california-lawn-detection/assets/Cities2015")

# Filter for LA City
la_cities = cities.filter(ee.Filter.eq('County', 'Los Angeles'))

la_county_income_zipcode2 = ee.FeatureCollection("projects/california-lawn-detection/assets/income_zipcode2019")
la_county_income_zipcode = la_county_income_zipcode2.select(ee.List(['zipcode', '2019zipcod','shape_area']), ee.List(['ZipCode', 'Median_Income','Area_sqft']))

In [5]:
Map = geemap.Map()

In [6]:
Map.addLayer(la_county_income_zipcode,{}, 'la_county_zipcode')

In [7]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [8]:
image_collection_2020_modis = (
    ee.ImageCollection('MODIS/006/MOD11A2')
    .filterDate('2020-04-01', '2020-06-30')
    .select(['LST_Day_1km', 'LST_Night_1km'])
)

In [9]:
image_2020_modis = image_collection_2020_modis.median()

image_2020_modis_clipped = image_2020_modis.clip(la_county)

daytime = image_2020_modis_clipped.select('LST_Day_1km')

landSurfaceTemperatureVis = {
  'min': 14000.0,
  'max': 16000.0,
  'palette': [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
  ],
}


In [10]:
Map.addLayer(image_2020_modis_clipped,{}, 'image_2020_modis_clipped')

Map.addLayer(daytime, landSurfaceTemperatureVis, 'Land Surface Temperature')

In [12]:
z90802 = la_county_income_zipcode.filter(ee.Filter.eq('ZipCode', '90802')) 

In [25]:
def mean_calculation_daytime(image, shape, pixel_scale):
    
    areaImage = image.select('LST_Day_1km')
    
    reducers = ee.Reducer.mean().combine({'reducer2': ee.Reducer.stdDev(),'sharedInputs': True})
    
    mean_temperature = areaImage.reduceRegion(
        reducer = reducers,
        geometry = shape.geometry(), 
        scale = pixel_scale, 
        maxPixels = 1e13)
    
    return mean_temperature

In [26]:
t = mean_calculation(image_2020_modis_clipped, z90802, 1000)

In [32]:
print(t.getInfo().get('LST_Day_1km') * .02 - 273)

33.498711624245516


In [ ]:
Temperatura de dia y temperatura de noche

In [17]:
Map

Map(bottom=26344.0, center=[34.615126683462194, -117.84484863281251], controls=(WidgetControl(options=['positi…

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import geemap

Map = geemap.Map()

# Combine the mean and standard deviation reducers.
reducers = ee.Reducer.mean().combine({
  'reducer2': ee.Reducer.stdDev(),
  'sharedInputs': True
})

# Use the combined reducer to get the mean and SD of the image.
stats = image.reduceRegion({
  'reducer': reducers,
  'bestEffort': True,
})
Map